# FEM - r adativity
Por Danilo Aballay, Vicente Iligaray, Ignacio Tapia y Manuel Sánchez


\begin{align*}
\nabla\cdot(\sigma\nabla u ) &= f\\
u &= 0 \quad \text{at }\Gamma_D\\
\nabla u\cdot \mathbf{n} &= g_N \quad \text{at }\Gamma_N\\
\end{align*}

## 1D Denso

### Benchmark 1

\begin{align*}
u &= x^{0.7}\\
f &= 0.21x^{-1.3}\\
u(0) &= 0\\
u'(1) &= 0.7\\
\sigma &= 1
\end{align*}

<p align="center">
    <img src="codes/Figures/x07_grid_comparison16_iter10000.png" alt="Optimización de la grilla para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
    <img src="codes/Figures/x07_r_adaptivity_sparse_nodes16_iter10000.png" alt="Resultado para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
</p>
<p align="center">
    <img src="codes/Figures/x07_Loss_history16_iter10000.png" alt="Historial de pérdida para $f(x) = x^{0.7}" width="45%" style="display: inline-block;">





### Benchmark 2

\begin{align*}
f &= 4\pi^2\sin(2\pi x)\\
u(0) &= 0\\
u(1) &= 0\\
\sigma &= \begin{cases}1 & x\leq \frac{1}{2}\\ 10 & x\geq \frac{1}{2}
\end{cases}
\end{align*}


<p align="center">
    <img src="codes/Figures/s_dis_grid_comparison17_iter10000.png" alt="Optimización de la grilla para sigma discontinuo" width="45%" style="display: inline-block;">
    <img src="codes/Figures/s_dis_r_adaptivity_sparse_nodes17_iter10000.png" alt="Resultado para sigma discontinuo" width="45%" style="display: inline-block;">
</p>
<p align="center">
    <img src="codes/Figures/s_dis_Loss_history17_iter10000.png" alt="Historial de pérdida para sigma discontinuo" width="45%" style="display: inline-block;">

## Convergencia FEM malla uniforme v/s FEM malla NN

Se evaluó la convergencia del Ritz utilizando la siguiente función:

$$ x^\alpha = 0 $$

Se utilizaron nodos basados en la potencia de dos, entre $2^2$ hasta $2^8$. Para cada uno de los escenarios, se grafíca cómo disminuye el Ritz en función de la cantidad de nodos utilizados, para luego calcular la pendiente de la curva de convergencia.


#### Caso 1 : $\alpha = 0.7$ 

<div style="text-align: center;">
    <img src="codes/Figures/NNlogN0.7_adaptive.png" alt="title"  width="60%">
</div>

**Tasa de convergencia:**
+ FEM malla uniforme: ~0.2
+ FEM malla NN: ~1.0

#### Caso 2: $\alpha = 0.7$

<div style="text-align: center;">
    <img src="codes/Figures/NNlogN0.7_adaptive.png" alt="title"  width="60%">
</div>

**Tasa de convergencia:**
+ FEM malla uniforme: ~0.2
+ FEM malla NN: ~1.0

#### Caso 3: $\alpha = 0.75$

<div style="text-align: center;">
    <img src="codes/Figures/NNlogN0.75_adaptive.png" alt="title"  width="60%">
</div>

**Tasa de convergencia:**
+ FEM malla uniforme: ~0.25
+ FEM malla uniforme teórica: 0.25
+ FEM malla NN: ~1.0
+ FEM malla óptima: 1.0

#### Caso 4: $\alpha = 0.65$

<div style="text-align: center;">
    <img src="codes/Figures/NNlogN0.65_adaptive.png" alt="title"  width="70%">
</div>

**Tasa de convergencia**:
+ FEM malla uniforme: ~0.15
+ FEM malla uniforme teórica: 0.15
+ FEM malla NN: ~1.0
+ FEM malla óptima teórica: 1.0





## Implementación Sparse 1D

Se implementó una versión del código para manejar la matriz de rigidez en formato *sparse*, con el objetivo de utilizar el solver lineal de JAX (`jax.experimental.sparse.spsolve`) y hacer el problema escalable. La dificultad de esta tarea radica en el reciente desarrollo de este solver, la limitada documentación disponible y la complejidad de manipular matrices en formato *sparse*.

Para realizarlo, se hicieron los siguientes pasos generales:

1. Se desarrolló una función para escribir la matriz de rigidez directamente en formato COO. Para ello, se definieron *arrays* con información de los elementos no cero, así como sus respectivas filas y columnas.

2. Se creó una función para transformar una matriz en formato COO a CSR, ya que el solver requiere este formato. A diferencia de COO, donde los elementos en la misma posición se suman, CSR necesita un único elemento por ubicación. Por lo tanto, se extrajo la información de la matriz COO, se preprocesó para identificar elementos duplicados y luego se definió una matriz CSR.

La ventaja de esta implementación es la capacidad de escalar la red neuronal para utilizar una mayor cantidad de nodos. Esto es particularmente relevante para sistemas 2D y 3D.

Para visualizar la ventaja de esta versión de **FEM-r adaptivity** usando el formato *sparse*, se optimizó la grilla para el primer ejemplo, cuya función es $f(x) = x^{0.7}$, utilizando 10000 nodos. 
<p align="center">
    <img src="codes\Figures\x07_r_adaptivity_sparse_nodes10000_iter1000.png" alt="Optimización de la grilla para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
    <img src="codes\Figures\x07_Loss_history10000_iter1000.png" alt="Historial de pérdida para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
</p>


El entrenamiento tardó 49.58 segundos, mientras que si se ejecuta este mismo entrenamiento con el formato denso arroja un error por falta de memoria para computar la solución del FEM:

<p align="center">
    <img src="codes\Figures\Error_dense_10000_nodes.png" alt="Error al entrenar sistema denso con 10000 nodos" width="80%" style="display: inline-block;">

## Ejemplos en 2D

### Benchmark 1

\begin{align*}
f &= -200(\arctan(10y - 1/2) - \arctan(1/2))\frac{(1/2-10x)}{(1+(10x-1/2)^2)^2} - 200(\arctan(10x - 1/2) - \arctan(1/2))\frac{(1/2-10y)}{(1+(10y-1/2)^2)^2}\\
g_N &= \begin{cases}(\arctan(10x - 1/2) + \arctan(1/2))\frac{10}{1+(10 - 1/2)^2}& \text{at }y = 1,\\
(\arctan(10y - 1/2) + \arctan(1/2))\frac{10}{1+(10 - 1/2)^2}& \text{at }x = 1.\\\end{cases}\\
\end{align*}


<p align="center">
    <img src="codes/Figures/Restult_2Dtest2.png" alt="Optimización de la grilla para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
    <img src="codes/Figures/LossHistory_2DTest2.png" alt="Resultado para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
</p>



### Benchmark 2

\begin{align*}
f &= -200(\arctan(10y - 2) - \arctan(2))\frac{(5-10x)}{(1+(10x-5)^2)^2} - 200(\arctan(10x - 5) - \arctan(5))\frac{(2-10y)}{(1+(10y-2)^2)^2}\\
g_N &= \begin{cases}(\arctan(10x - 5) + \arctan(5))\frac{10}{1+(8)^2}& \text{at }y = 1,\\
(\arctan(10y - 2) + \arctan(2))\frac{10}{1+(5)^2}& \text{at }x = 1.\\\end{cases}\\
\end{align*}

<p align="center">
    <img src="codes/Figures/Results_2DTest2_disp.png" alt="Optimización de la grilla para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
    <img src="codes/Figures/LossHistory_2Dtest2_disp.png" alt="Resultado para $f(x) = x^{0.7}$" width="45%" style="display: inline-block;">
</p>

## Red neuronal $\sigma$ - FEM r adaptivity

Se implementó una red neuronal que tuviese el objetivo de aprender la relación entre el coeficiente de difusión del sistema y la grilla óptima tal de obtener el mínimo Ritz de la solución del FEM. El sistema utilizado para entrenar la red fue el siguiente:

$$ -\nabla (\sigma \nabla \mu) = f $$
$$ \mu(x) = 0, \quad x=0 \text{ y } x=1 $$
$$ f(x) = 4\pi^2 \sin(2\pi x) $$
$$ \sigma(x) = m_\sigma + 0.1$$

Para ello, la red neuronal tiene la siguiente estructura:

- **Input**: Parámetro $m_\sigma$, utilizando un set de 100 puntos equiespaciado entre 0.1 y 100.
- **Hidden layers**: 2 capas con 10 neuronas cada una. 
- **Función de activación**: $\tanh$.
- **Output**: Capa sin activación con dimensión de 16, equivalente a las distancias entre nodos en la grilla luego de aplicar softmax.
- **Función de costos**: sumatoria de los Ritz para cada $m_x$ usado en el entrenamiento.
- **Learning rate**: $1e-3$.
- **Optimizador Adam**
- **10000 iteraciones**

A continuación se muestra el resultado de la red considerando el valor de $m_\sigma$ menor y mayor, iguales a 0.1 y 100, respectivamente.

<p align="center">
    <img src="codes\Figures\NN_sigma.png" alt="Resultado NN" width="80%" style="display: inline-block;">
</p>


Se observa que para cada valor de $m_\sigma$, la red aprendió la forma de minimizar el Ritz modificando la posición de los nodos. En la figura se puede apreciar cómo la red neuronal ajusta la distribución de los nodos para adaptarse a las variaciones en el coeficiente de difusión $\sigma$. Para valores altos de $m_\sigma$, los nodos se concentran en los extremos, mientras que para valores pequeños de $m_\sigma$, los nodos se distribuyen de manera más uniforme.

Esto indica que la red neuronal es capaz de capturar la relación entre el coeficiente de difusión y la distribución óptima de los nodos, permitiendo obtener una grilla adaptativa que minimiza el error de Ritz para cada función lineal de $\sigma$ en un problema FEM. Cabe destacar que, gracias a la implementación *sparse* del modelo 1D, es posible entrenar con alta eficiencia y se puede escalar para sistemas más grandes y complejos.

En resumen, la red neuronal no solo ofrece una solución eficiente para problemas FEM en 1D, sino que también tiene el potencial de ser aplicada a problemas en dimensiones superiores, proporcionando una herramienta poderosa para la optimización de grillas en sistemas de mayor complejidad.